In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
import pandas as pd

In [2]:
#차트 페이지 열기
browser = webdriver.Chrome('C:\selenium\chromedriver.exe')
browser.get('http://www.gaonchart.co.kr/')
browser.find_element_by_xpath('//*[@id="mainwrap"]/div[1]/div[1]/div[1]/div[1]/div/a').click()

In [3]:
#차트 크롤링할 함수 선언
def chart_crawler():
    #데이터프레임 준비
    gaon_chart = pd.DataFrame({
    '순위':[],
    '변동':[],
    '제목':[],
    '가수':[],
    '앨범':[],
    '가온지수':[],
    '제작사':[],
    '유통사':[]
    })

    #순위
    ranks = browser.find_elements_by_xpath('//td[@class="ranking"]')

    #변동(beautiful soup 이용)
    req = requests.get(browser.current_url)
    soup = BeautifulSoup(req.content, 'html.parser')
    changes=[]
    for i in soup.select('.change'):
        if i.select('.up'):
            if i.select('.up')=='HOT':
                changes.append('HOT')
            else:
                changes.append(i.get_text()+' 상승')
        elif i.select('.down'):
            changes.append(i.get_text()+' 하락')
        elif i.select('.new'):
            changes.append('NEW')
        else:
            changes.append('변동 없음')


    #제목
    titles = browser.find_elements_by_xpath('//td[@class="subject"]/p[1]')

    #가수 & 앨범
    singers_n_albums = browser.find_elements_by_xpath('//p[@class="singer"]')

    #가온지수
    gaon_count = browser.find_elements_by_xpath('//td[@class="count"]')

    #제작사
    production = browser.find_elements_by_xpath('//p[@class="pro"]')

    #유통사
    distribution = browser.find_elements_by_xpath('//p[@class="dist"]')

    #가수와 앨범명 분리
    singers = []
    albums = []
    for n in singers_n_albums:
        if len((n.text).split('|'))==2:
            singers.append((n.text).split('|')[0])
            albums.append((n.text).split('|')[1])
        else:
            singers.append((n.text))
            albums.append(np.nan)

    #text 추출
    for i in range(200):
        ranks[i] = ranks[i].text
        titles[i] = titles[i].text
        gaon_count[i] = gaon_count[i].text
        production[i] = production[i].text
        distribution[i] = distribution[i].text
         
    #데이터프레임 만들기
    insert_data = pd.DataFrame({
        '순위':ranks,
        '변동':changes,
        '제목':titles,
        '가수':singers,
        '앨범':albums,
        '가온지수':gaon_count,
        '제작사':production,
        '유통사':distribution
    })

    gaon_chart = gaon_chart.append(insert_data)
    gaon_chart.set_index('순위', inplace=True)
    
    # 현재 열려있는 탭 종료
    browser.close()

    # 브라우저 전체 종료
    browser.quit()
    
    return gaon_chart


In [4]:
#차트 크롤링
gaon_chart = chart_crawler()

In [5]:
gaon_chart

,변동,제목,가수,앨범,가온지수,제작사,유통사
순위,,,,,,,
1,변동 없음,Stay,"The Kid LAROI, Justin Bieber",Stay,"31,202,952",Columbia,Sony Music
2,변동 없음,신호등,이무진,신호등,"26,726,205",쇼플레이,Kakao Entertainment
3,변동 없음,Next Level,aespa,Next Level,"21,282,805",SM Entertainment,Dreamus
4,변동 없음,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집,"20,292,771",유야호,"지니뮤직, Stone Music Entertainment"
5,변동 없음,Weekend,태연 (TAEYEON),Weekend,"19,536,190",SM Entertainment,Dreamus
...,...,...,...,...,...,...,...
196,변동 없음,좋은 사람 있으면 소개시켜줘,조이 (JOY),슬기로운 의사생활 OST Part 2,"2,052,091","스튜디오 마음C, 에그이즈커밍, Stone Music Entertainment","지니뮤직, Stone Music Entertainment"
197,62 하락,눈물을 훔친다 (금혼령 X 김종국),김종국,눈물을 훔친다 (금혼령 X 김종국) OST,"2,003,983",TOON STUDIO,"지니뮤직, Stone Music Entertainment"
198,변동 없음,Believer,Imagine Dragons,Evolve,"2,000,505","KIDinaKORNER, Interscope Records",Universal Music


In [6]:
#결측치 확인
gaon_chart.isnull().sum()

변동      0
제목      0
가수      0
앨범      2
가온지수    0
제작사     0
유통사     0
dtype: int64

In [7]:
#앨범 결측치 확인
gaon_chart['앨범'][gaon_chart['앨범'].isnull()]

순위
69    NaN
93    NaN
Name: 앨범, dtype: object

In [8]:
#비어있는 부분 확인
gaon_chart.loc[['69','93'],['가수','앨범']]

,가수,앨범
순위,,
69,"MSG워너비 TOP 8 (별루지, 김정수, 강창모, 정기석, 이동휘, 이상이, 박재...",NaN
93,"미란이 (Mirani), 먼치맨, Khundi Panda, 머쉬베놈 (MUSHVEN...",NaN


In [9]:
#결측치를 채워줄 함수 선언
def missing_data(gaon_chart):
    #앨범 결측치 확인
    album_loss = gaon_chart['앨범'][gaon_chart['앨범'].isnull()]

    #해당되는 앨범의 곡명 추출
    loss_list=[]
    loss_singers=[]
    for i in range(len(album_loss)):
        loss_list.append(gaon_chart['제목'][album_loss.index[i]])
        loss_singers.append(gaon_chart['가수'][album_loss.index[i]])
    
    #곡의 가수와 앨범명 찾기
    browser2 = webdriver.Chrome('C:\selenium\chromedriver.exe')
    browser2.get('https://www.naver.com/')
    albums_new=[]
    singers_new=[]
    for i in range(len(loss_list)):
        search = browser2.find_element_by_xpath('//*[@id="query"]')
        search.send_keys(loss_list[i]+loss_singers[i])
        search.submit()
        albums_new.append(browser2.find_element_by_xpath('//div[@class="dsc_area"]/span[1]').text)
        singers_new.append(browser2.find_element_by_xpath('//div[@class="dsc_area"]/span[2]').text)
        browser2.back()
    
    # 현재 열려있는 탭 종료
    browser2.close()

    # 브라우저 전체 종료
    browser2.quit()
    
    #결측치 채우기
    for i in range(len(loss_list)):
        gaon_chart.loc[album_loss.index[i], '앨범'] = albums_new[i]
        gaon_chart.loc[album_loss.index[i], '가수'] = singers_new[i]
    
    return gaon_chart

In [10]:
#결측치 채우기
gaon_chart = missing_data(gaon_chart)

In [11]:
#결측치 확인
gaon_chart.isnull().sum()

변동      0
제목      0
가수      0
앨범      0
가온지수    0
제작사     0
유통사     0
dtype: int64

In [12]:
#채워진 부분 확인
gaon_chart.loc[['69','93'],['가수','앨범']]

,가수,앨범
순위,,
69,"MSG워너비 TOP 8(별루지, 김정수, 강창모, 정기석, 이동휘, 이상이, 박재정...",MSG워너비 TOP 8 경연곡
93,"미란이(Mirani), 먼치맨(MUNCHMAN), 쿤디판다(Khundi Panda)...",쇼미더머니 9 Episode 1


In [13]:
gaon_chart

,변동,제목,가수,앨범,가온지수,제작사,유통사
순위,,,,,,,
1,변동 없음,Stay,"The Kid LAROI, Justin Bieber",Stay,"31,202,952",Columbia,Sony Music
2,변동 없음,신호등,이무진,신호등,"26,726,205",쇼플레이,Kakao Entertainment
3,변동 없음,Next Level,aespa,Next Level,"21,282,805",SM Entertainment,Dreamus
4,변동 없음,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집,"20,292,771",유야호,"지니뮤직, Stone Music Entertainment"
5,변동 없음,Weekend,태연 (TAEYEON),Weekend,"19,536,190",SM Entertainment,Dreamus
...,...,...,...,...,...,...,...
196,변동 없음,좋은 사람 있으면 소개시켜줘,조이 (JOY),슬기로운 의사생활 OST Part 2,"2,052,091","스튜디오 마음C, 에그이즈커밍, Stone Music Entertainment","지니뮤직, Stone Music Entertainment"
197,62 하락,눈물을 훔친다 (금혼령 X 김종국),김종국,눈물을 훔친다 (금혼령 X 김종국) OST,"2,003,983",TOON STUDIO,"지니뮤직, Stone Music Entertainment"
198,변동 없음,Believer,Imagine Dragons,Evolve,"2,000,505","KIDinaKORNER, Interscope Records",Universal Music


In [14]:
#csv 파일로 저장
gaon_chart.to_csv('gaon_chart.csv')

In [15]:
#다시 불러와서 확인
gaon_chart2 = pd.read_csv('gaon_chart.csv', index_col=0)
gaon_chart2

,변동,제목,가수,앨범,가온지수,제작사,유통사
순위,,,,,,,
1,변동 없음,Stay,"The Kid LAROI, Justin Bieber",Stay,"31,202,952",Columbia,Sony Music
2,변동 없음,신호등,이무진,신호등,"26,726,205",쇼플레이,Kakao Entertainment
3,변동 없음,Next Level,aespa,Next Level,"21,282,805",SM Entertainment,Dreamus
4,변동 없음,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집,"20,292,771",유야호,"지니뮤직, Stone Music Entertainment"
5,변동 없음,Weekend,태연 (TAEYEON),Weekend,"19,536,190",SM Entertainment,Dreamus
...,...,...,...,...,...,...,...
196,변동 없음,좋은 사람 있으면 소개시켜줘,조이 (JOY),슬기로운 의사생활 OST Part 2,"2,052,091","스튜디오 마음C, 에그이즈커밍, Stone Music Entertainment","지니뮤직, Stone Music Entertainment"
197,62 하락,눈물을 훔친다 (금혼령 X 김종국),김종국,눈물을 훔친다 (금혼령 X 김종국) OST,"2,003,983",TOON STUDIO,"지니뮤직, Stone Music Entertainment"
198,변동 없음,Believer,Imagine Dragons,Evolve,"2,000,505","KIDinaKORNER, Interscope Records",Universal Music
